In [5]:
import datetime
import QuantExt as qe
from SWAP_CURVE_hb import GET_QUOTE, USDIRS_CURVE, KRWCCS_CURVE

In [15]:
class CCS():
    # ccs_rate : 원화 고정금리 (거래시점에 결정됨)
    # fx_spot : 시장환율
    def __init__(self, todays_date, effective_date, maturity_date, ccs_rate, fx_spot, usd_notioanl, position):
        #date / curve / fx_spot rate
        self.date = todays_date
        self.usd_curve = self.USD_CURVE(self.date)
        self.krw_curve = self.KRW_CURVE(self.date)
        self.fx_spot = fx_spot
        
        #CCS에 대한 시장정보
        self.effective_date = qe.Date(effective_date.day, effective_date.month, effective_date.year)
        self.maturity_date = qe.Date(maturity_date.day,maturity_date.month,maturity_date.year)
        
        if position == 'Long':
            self.position = qe.VanillaSwap.Payer  #krw고정금리지급
        else:
            self.position = qe.VanillaSwap.Receiver #krw고정금리 수취
        
        self.ccs_rate = ccs_rate
        self.usd = qe.USDCurrency()
        self.usd_notional = usd_notional
        self.krw = qe.KRWCurrency()
        self.krw_notional = usd_notional * fx_spot #환율곱해주기
        
        self.length = 2 #2일 간격
        self.spread = 0.0
        self.convention = qe.ModifiedFollowing
        self.calendar = qe.JointCalendar(qe.SouthKorea(), qe.UnitedStates())
        self.tenor = qe.Period(6, qe.Months)
        
        self.fixed_day_count = qe.Actual365Fixed() #원화고정금리
        self.fload_day_count = qe.Actual360() #달러변동금리 
        self.dateGeneration = qe.DateGeneration.Backward
        
        self.npv = self.PRICING(self.usd_curve, self.krw_curve, self.fx_spot)
        self.fx_delta = self.FX_DELTA()
        self.usd_ir_delta = self.USD_IR_DELTA()
        self.krw_ir_delta = self.KRW_IR_DELTA()
        self.theta = self.THETA()
        
    def USD_CURVE(self, date): 
        return USDIRS_CURVE(date, GET_QUOTE(date, 'USD'))
    
    def KRW_CURVE(self, date):
        return KRWCCS_CURVE(date, GET_QUOTE(date, 'KRW'))
    
    def PRICING(self, usd_curve, krw_curve, fx_spot):
        usd_curve_handle = qe.YieldTermStructureHandle(usd_curve)
        krw_curve_handle = qe.YieldTermStructureHandle(krw_curve)
        fx_spot_handle=  qe.QuoteHandle(qe.SimpleQuote(fx_spot))
        
        usd_6m_libor = qe.USDLibor(qe.Period(6, qe.Months),
                                   usd_curve_handle)
        usd_6m_libor.addFixing(qe.Date(24,9,2020), 0.00097)
        
        #fixed schedule
        fixed_schedule = qe.Schedule(self.effective_date,
                                    self.maturity_date,
                                    self.tenor,
                                    self.calendar,
                                    self.convention, #중간중간 이자지급 휴일처리
                                    self.convention, #만기일자 휴일처리방식
                                    self.dateGeneration,
                                    False) #endofmonth
        
        float_schedule = qe.Schedule(self.effective_date,
                                    self.maturity_date,
                                    self.tenor,
                                    self.calendar,
                                    self.convention, #중간중간 이자지급 휴일처리
                                    self.convention, #만기일자 휴일처리방식
                                    self.dateGeneration,
                                    False) #endofmonth
        
        #ccs
        ccs = qe.CrossCcyFixFloatSwap(self.position,
                                     self.krw_notional,
                                     self.krw,
                                     fixed_schedule,
                                     self.ccs_rate,
                                     self.fixed_day_count,
                                     self.convention,
                                     self.length,
                                     self.calendar,
                                     self.usd_notional,
                                     self.usd,
                                     float_schedule,
                                     usd_6m_libor,
                                     self.spread,
                                     self.convention,
                                     self.length,
                                     self.calendar)
        
        engine = qe.CrossCcySwapEngine(self.krw,
                                      krw_curve_handle,
                                      self.usd,
                                      usd_curve_handle,
                                      fx_spot_handle)
        ccs.setPricingEngine(engine)
        npv = ccs.NPV()
        return npv
    
    def FX_DELTA(self):
        # 1%
        percentage = 0.01

        # fx forward price when 1% up
        up_fx = self.fx_spot * (1 + percentage)
        up_fxf = self.PRICING(self.usd_curve, self.krw_curve, up_fx)

        # fx forward price when 1% down
        down_fx = self.fx_spot * (1 - percentage)
        down_fxf = self.PRICING(self.usd_curve, self.krw_curve, down_fx)

        fx_delta = (up_fxf - down_fxf) / 2
        return fx_delta
    
    def USD_IR_DELTA(self):
        # Handle of USD Curve
        curve_handle = qe.YieldTermStructureHandle(self.usd_curve)
        
        # 1bp
        basis_point = 0.0001
        
        # FX Forward price when 1bp up
        up_curve = qe.ZeroSpreadedTermStructure(curve_handle, qe.QuoteHandle(qe.SimpleQuote(basis_point)))
        up_fxf = self.PRICING(up_curve, self.krw_curve, self.fx_spot)
        
        # FX Forward price when 1bp down
        down_curve = qe.ZeroSpreadedTermStructure(curve_handle, qe.QuoteHandle(qe.SimpleQuote(-basis_point)))
        down_fxf = self.PRICING(down_curve, self.krw_curve, self.fx_spot)

        # USD Curve Delta
        delta = (up_fxf - down_fxf) / 2
        return delta
    
    def KRW_IR_DELTA(self):
        # Handle of KRW Curve
        curve_handle = qe.YieldTermStructureHandle(self.krw_curve)
        
        # 1bp
        basis_point = 0.0001
        
        # FX Forward price when 1bp up
        up_curve = qe.ZeroSpreadedTermStructure(curve_handle, qe.QuoteHandle(qe.SimpleQuote(basis_point)))
        up_fxf = self.PRICING(self.usd_curve, up_curve, self.fx_spot)
        
        # FX Forward price when 1bp down
        down_curve = qe.ZeroSpreadedTermStructure(curve_handle, qe.QuoteHandle(qe.SimpleQuote(-basis_point)))
        down_fxf = self.PRICING(self.usd_curve, down_curve, self.fx_spot)

        # KRW Curve Delta
        delta = (up_fxf - down_fxf) / 2
        return delta 
    
    def THETA(self):
        # FX Forward Price at T0
        price_t0 = self.PRICING(self.usd_curve, self.krw_curve, self.fx_spot)

        # FX Forward Price at T1
        usd_curve_t1 = self.USD_CURVE(self.date + datetime.timedelta(days=1))
        krw_curve_t1 = self.KRW_CURVE(self.date + datetime.timedelta(days=1))
        price_t1 = self.PRICING(usd_curve_t1, krw_curve_t1, self.fx_spot)
        
        # Theta
        theta = price_t1 - price_t0
        return theta
    
if __name__ == '__main__':
    todays_date = datetime.date(2020,10,8)
    fx_spot = 1133.85 #시장환율
    #ccs거래정보
    effective_date = datetime.date(2020,9,26) #2일전에 변동금리결정일. #그래서 위에 9월24일로 addFixing
    maturity_date = datetime.date(2025, 9,26)
    position = 'Long'
    ccs_rate = 0.002276
    usd_notional = 10000000
    
    ccs = CCS(todays_date,
             effective_date,
             maturity_date,
             ccs_rate,
             fx_spot,
             usd_notional,
             position)
    # Print the Results
    print("Price = {}".format(round(ccs.npv, 4)))
    print("FX Delta = {}".format(round(ccs.fx_delta, 4)))
    print("USD IR Delta = {}".format(round(ccs.usd_ir_delta, 4)))
    print("KRW IR Delta = {}".format(round(ccs.krw_ir_delta, 4)))
    print("Theta = {}".format(round(ccs.theta, 4)))

Price = -51359034.2349
FX Delta = 113310697.5572
USD IR Delta = -544307.5481
KRW IR Delta = 5640421.6167
Theta = 36405.7949
